In [123]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression

In [124]:
data = pd.read_csv("EuroMillions_numbers.csv", sep=';')

In [125]:
def get_data(d):
    d = d.select_dtypes(np.number)
    d['HasWin'] = np.ones(d['Winner'].size, dtype=int)
    d = d.drop('Gain', 1)
    d = d.drop('Winner', 1)
    false_data = add_false_data(d)
    return pd.concat([d, false_data])

def add_false_data(d):
    false_data = d.copy()
    for index, row in false_data.iterrows():
        for i in range(5):
            row[i] = np.random.uniform(0, 50)
        for i in range(2):
            row[i + 5] = np.random.uniform(0, 12)
        row[7] = 0
    # Enlever les doublons plus tard    
    return false_data

In [126]:
combined_data = get_data(data)
train_data = combined_data.sample(frac = 0.5) #bizarre
test_data = combined_data.drop(train_data.index)
test_data = [test_data.drop('HasWin', 1), test_data[['HasWin']]]

y = train_data['HasWin'].astype('category').cat.codes
x = train_data.select_dtypes(np.number).drop('HasWin', 1)

modele_logit = LogisticRegression(penalty='none')
modele_logit.fit(x,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='none',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [127]:
pd.DataFrame(np.concatenate([modele_logit.intercept_.reshape(-1, 1), modele_logit.coef_],axis=1),
             index = ["coef"],
             columns = ["constante"]+list(x.columns)).T


,coef
constante,-1.026040
N1,-0.010676
N2,-0.002488
N3,0.007008
N4,0.009221
N5,0.015853
E1,-0.030427
E2,0.105271


In [128]:

y_pred = modele_logit.predict (test_data[0])
sum = 0
for i in range (len(y_pred)):
    if (y_pred[i] == test_data[1].values.tolist()[i][0]): sum += 1
print(sum / len(y_pred))

      

0.6215384615384615
